<a href="https://colab.research.google.com/github/karthik-palaniappan/Mini-Project-Speech-Emotion-Classification/blob/main/T7_Copy_of_M2_Supplementary_Audio_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Supplementary: Audio feature extraction

**Note**: This notebook is an ungraded, supplementary notebook, provided as pre-reading for the 'Speech Emotion Recognition' Mini-project. This will help you to understand the various audio features and get introduced to the tools and packages used in audio feature extraction.

## Learning Objectives

In this supplementary notebook, you will be able to:

* record audio samples and save it
* load the audio data from the dataset
* extract the features of the audio data

### Dataset

The [SAVEE](http://kahlan.eps.surrey.ac.uk/savee/) database was recorded from four native English male speakers (identified as DC, JE, JK, KL), postgraduate students and researchers at the University of Surrey aged from 27 to 31 years. It contains 480 British English utterances in different emotions. Emotion has been described psychologically in discrete categories: anger, disgust, fear, happiness, sadness and surprise.'Neutral' emotion was added to provide recordings of 7 emotion categories. The text material consisted of 15 sentences per emotion: 3 common, 2 emotion-specific and 10 generic sentences that were different for each emotion and phonetically-balanced. The 3 common and 2 × 6 = 12 emotion-specific sentences were recorded as neutral to give 30 neutral sentences.

  - This resulted in a total of 120 utterances per speaker, for example:

      - Common: She had your dark suit in greasy wash water all year.
      - Anger: Who authorized the unlimited expense account?
      - Disgust: Please take this dirty table cloth to the cleaners for me.
      - Fear: Call an ambulance for medical assistance.
      - Happiness: Those musicians harmonize marvelously.
      - Sadness: The prospect of cutting back spending is an unpleasant one for any governor.
      - Surprise: The carpet cleaners shampooed our oriental rug.
      - Neutral: The best way to learn is to solve extra problems.

### Download the dataset

In [ ]:
!wget -qq https://cdn.iisc.talentsprint.com/CDS/MiniProjects/SaveeData.zip
!unzip -qq SaveeData.zip

In [ ]:
# Install packages
!pip -qq install librosa soundfile

### Import required packages

In [ ]:
import librosa
import librosa.display
import soundfile
import re
import os, glob, pickle
import numpy as np
from matplotlib import pyplot as plt
import IPython.display as ipd
from datetime import datetime
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

### Record Audio sample

We will use the Javascript from IPython that will enable us to record the audio.

* Create a directory to store all the recordings
* Naming convention of the audio sample is `Audio_DateTime_label.wav`, For eg, *Audio_2021-06-20_11-30-30_angry.wav*
* Upon calling the function `record()`, it prompts to enter the emotion (label) to name the file.

While recording the audio, the utterance can be "*Say the word boat*" and you can show variation in the emotion

In [ ]:
# Create a directory
os.mkdir("Records")

In [ ]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
    emt = input("Enter emotion(label) to save the recording: ")
    print("Start speaking!")
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d_%H-%M-%S")
    display(Javascript(RECORD))
    s = output.eval_js('record(%d)' % (sec*1000))
    b = b64decode(s.split(',')[1])
    with open('Records/audio_'+current_time+"_"+emt+'.wav','wb') as f:
      f.write(b)
    return 'Records/audio_'+current_time+"_"+emt+'.wav'

In [ ]:
# call the function to record a sample
record()

In [ ]:
# change the path and listen to recorded audio sample
#ipd.Audio("SaveeData/JE/a08.wav")

### Load the dataset

In [ ]:
# The glob module is used to retrieve files/pathnames matching a specified pattern.
# It is faster than other methods to match pathnames in directories
wav_files = glob.glob("SaveeData/*/*.wav")
len(wav_files)

### Explore the data and visualize

#### Various emotions in given dataset

In [ ]:
wav_files[0]

In [ ]:
emotions_ = []
mapping = {'a': 'angry', 'f': 'fear', 'd': 'disgust', 'h': 'happiness', 'sa': 'sadness', 'su': 'surprise', 'n': 'neutral'}
for file_name in wav_files:
    emotions_.append(mapping[re.sub('[0-9]*', '', file_name.split("/")[-1].split(".")[0])])
set(emotions_)

####Speech/Audio analysis Python Package: **Librosa**

[Librosa](https://librosa.org/doc/latest/index.html) is a Python package for audio and music signal processing. At a high level, librosa provides implementations of a variety of common functions and helps in audio feature extraction and manipulation, segmentation, visualization, and display of feature representations.

#### Visualize sample audio signal using librosa

In [ ]:
sample_audio_path = '/content/SaveeData/DC/a03.wav'

# librosa is used for analyzing and extracting features of an audio signal
data, sampling_rate = librosa.load(sample_audio_path)
plt.figure(figsize=(15, 5))

# librosa.display.waveshow is used to plot waveform of amplitude vs time
librosa.display.waveshow(data, sr=sampling_rate)
plt.show()

### Audio Features



### Extract features

* Load an audio using `librosa.load()`

    `load()` function loads an audio file and decodes it into a 1-dimensional array which is a time series x , and sr is a sampling rate of x . Default sr is 22kHz.


* Apply short term Fourier transform using `librosa.stft()

    `stft()` converts data into short term Fourier transform. STFT converts signal such that we can know the amplitude of given frequency at a given time. Using STFT we can determine the amplitude of various frequencies playing at a given time of an audio signal.


* Apply Mel-Frequency Cepstral Coefficients (MFCC) using librosa.feature.mfcc

    This feature is one of the most important method to extract a feature of an audio signal and is used majorly whenever working on audio signals. The mel frequency cepstral coefficients (MFCCs) of a signal are a small set of features (usually about 10–20) which concisely describe the overall shape of a spectral envelope.
    

* Compute a chromagram from a waveform or power spectrogram using librosa.feature.chroma_stft
    
    Chroma features are a powerful tool for analyzing audio whose pitches can be meaningfully categorized (often into twelve categories) and whose tuning approximates to the equal-tempered scale. One main property of Chroma features is that they capture harmonic and melodic characteristics of music, while being robust to changes in timbre and instrumentation.


* Compute a Mel-scaled spectrogram using librosa.feature.melspectrogram
    
    A Mel spectrogram is a spectrogram where the frequencies are converted to the Mel scale. Mel scale is a scale of pitches judged by listeners to be equal in distance one from another.

For more information related to audio based features, refer [here](https://heartbeat.fritz.ai/working-with-audio-signals-in-python-6c2bd63b2daf).

In [ ]:
def extract_feature(file_name):
    # load audio
    X, sample_rate = librosa.load(file_name)
    # apply stft()
    stft=np.abs(librosa.stft(X))
    result=np.array([])
    # compute mfcc
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    # combine the features
    result=np.hstack((result, mfccs))
    # compute chroma features and combine
    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    result=np.hstack((result, chroma))
    # compute melspectrogram and combine
    mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
    result=np.hstack((result, mel))
    return result

In [ ]:
sample_feature = extract_feature(wav_files[0]) #,mfcc=True, chroma=True, mel=True)
sample_feature.shape

In [ ]:
type(sample_feature)

### Extract features and labels of all audio samples

'DC', 'JE', 'JK' and 'KL' are four male speakers recorded for the SAVEE database. Audio files consist of audio WAV files..

There are 15 sentences for each of the 7 emotion categories.
The initial letter(s) of the file name represents the emotion class, and the following digits represent the sentence number.
The letters 'a', 'd', 'f', 'h', 'n', 'sa' and 'su' represent 'anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness' and 'surprise' emotion classes respectively.

E.g., '**SaveeData/JK/d03.wav**' is the 3rd disgust sentence.


In [ ]:
emotions_savee = {"a":0, "d":1, "f":2, "h":3, "n":4, "sa":5, "su":6}
emotions_savee

In [ ]:
# Declare list to append features and labels
features, labels = [], []

for wv_file in wav_files:
  features.append(extract_feature(wv_file))
  # extracting label
  emt = re.sub('[0-9]*', '', wv_file.split("/")[-1].split(".")[0])
  labels.append(emotions_savee[emt])

In [ ]:
len(features), len(labels)

In [ ]:
print(set(labels))

For further reading on audio acquisition, representation and storage refer the [book](http://www.dcs.gla.ac.uk/~vincia/textbook.pdf): *Camastra and Vinciarelli, 2007, Machine Learning for Audio, Image and Video Analysis*